# SageMaker 한국어 파인 튜닝 모델 평가 하기
- 이 노트북은 기존에 Llama3-8B 베이스 모델을 기반으로 파인 튜닝된 두개의 모델 (Quick-Model, Optimized-Model) 을 다운로드 받습니다. 
- 테스트 데이터 셋을 통하여 요약에 대한 평가지료를 측정 합니다.

## 0. 선수 내용
이 노트북은 아래의 노트북이 먼저 실행이 완료 되어 있어야만 합니다. 
- notebook/01-naver-news-fsdp-QLoRA/03-SageMaker-Training.ipynb
- notebook/01-naver-news-fsdp-QLoRA/03-1-Option-SageMaker-Training.ipynb

## 1. 환경 구성
01-naver-news-fsdp-QLoRA/03-SageMaker-Training.ipynb 노트북에서 model_s3_path 에 저장한 경로를 가져 옵니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = "../.."
add_python_path(module_path)

python path: /home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llama3 is added
sys.path:  ['/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llama3/notebook/01-naver-news-fsdp-QLoRA', '/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python310.zip', '/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10', '/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/lib-dynload', '', '/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages', '/home/ec2-user/SageMaker/huggingface-inferentia2-samples/llama3-70b/llmperf/src', '/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llama3']


In [2]:
%store -r full_test_data_json
print("full_test_data_json: ", full_test_data_json)

full_test_data_json:  ../data/naver-news-summarization-ko/full_test/test_dataset.json


## 2. 모델 아티펙트 다운로드 
이전에 훈련한 파인 튜닝 모델의 모델 아티펙트를 다운로드 합니다.
- 모델 다운로드는 아래 경로 입니다. 사용하시는 환경이 다르면 수정 하세요.
    - ```model_data_dir = "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news-s3-download" ```


#### Quick Version Model

In [3]:
import os

quick_model_data_dir = "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news-s3-quick"
os.makedirs(quick_model_data_dir, exist_ok=True)
print("quick_model_data_dir: ", quick_model_data_dir)

quick_model_data_dir:  /home/ec2-user/SageMaker/models/llama-3-8b-naver-news-s3-quick


In [4]:
%store -r model_s3_path
print("model_s3_path: ", model_s3_path)

quick_model_artifact_path = model_s3_path["S3DataSource"]["S3Uri"]
print("quick_model_artifact_path: ", quick_model_artifact_path)

model_s3_path:  {'S3DataSource': {'S3Uri': 's3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-00-52-1-2024-08-26-00-52-15-324/output/model/', 'S3DataType': 'S3Prefix', 'CompressionType': 'None'}}
quick_model_artifact_path:  s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-00-52-1-2024-08-26-00-52-15-324/output/model/


In [5]:
%%sh -s {quick_model_artifact_path} {quick_model_data_dir}

artifact_path=$1
model_data_dir=$2

echo $artifact_path
echo $model_data_dir

# 기존 데이터 삭제
rm -rf $model_data_dir/*

# 모델을 S3에서 로컬로 다운로드
aws s3 cp $artifact_path $model_data_dir --recursive

ls -al $model_data_dir



s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-00-52-1-2024-08-26-00-52-15-324/output/model/
/home/ec2-user/SageMaker/models/llama-3-8b-naver-news-s3-quick
download: s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-00-52-1-2024-08-26-00-52-15-324/output/model/generation_config.json to ../../../../../../../models/llama-3-8b-naver-news-s3-quick/generation_config.json
download: s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-00-52-1-2024-08-26-00-52-15-324/output/model/config.json to ../../../../../../../models/llama-3-8b-naver-news-s3-quick/config.json
download: s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-00-52-1-2024-08-26-00-52-15-324/output/model/model-00002-of-00007.safetensors to ../../../../../../../models/llama-3-8b-naver-news-s3-quick/model-00002-of-00007.safetensors
download: s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-00-52-1-2024-08-26-00-52-15-324/output/model

### Optimized Version Model

In [6]:
optimize_model_data_dir = "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news-s3-optimize"
os.makedirs(optimize_model_data_dir, exist_ok=True)
print("optimize_model_data_dir: ", optimize_model_data_dir)

optimize_model_data_dir:  /home/ec2-user/SageMaker/models/llama-3-8b-naver-news-s3-optimize


In [7]:
%store -r optimized_model_s3_path
print("optimized_model_s3_path: ", optimized_model_s3_path)

optimize_model_artifact_path = optimized_model_s3_path["S3DataSource"]["S3Uri"]
print("optimize_model_artifact_path: ", optimize_model_artifact_path)

optimized_model_s3_path:  {'S3DataSource': {'S3Uri': 's3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-02-52-1-2024-08-26-02-52-15-547/output/model/', 'S3DataType': 'S3Prefix', 'CompressionType': 'None'}}
optimize_model_artifact_path:  s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-02-52-1-2024-08-26-02-52-15-547/output/model/


In [8]:
%%sh -s {optimize_model_artifact_path} {optimize_model_data_dir}

artifact_path=$1
model_data_dir=$2

echo $artifact_path
echo $model_data_dir

# 기존 데이터 삭제
rm -rf $model_data_dir/*

# 모델을 S3에서 로컬로 다운로드
aws s3 cp $artifact_path $model_data_dir --recursive

ls -al $model_data_dir



s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-02-52-1-2024-08-26-02-52-15-547/output/model/
/home/ec2-user/SageMaker/models/llama-3-8b-naver-news-s3-optimize
download: s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-02-52-1-2024-08-26-02-52-15-547/output/model/config.json to ../../../../../../../models/llama-3-8b-naver-news-s3-optimize/config.json
download: s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-02-52-1-2024-08-26-02-52-15-547/output/model/generation_config.json to ../../../../../../../models/llama-3-8b-naver-news-s3-optimize/generation_config.json
download: s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-02-52-1-2024-08-26-02-52-15-547/output/model/model-00002-of-00007.safetensors to ../../../../../../../models/llama-3-8b-naver-news-s3-optimize/model-00002-of-00007.safetensors
download: s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-08-26-02-52-1-2024-08-26-02-52-15-547/

## 3. 모델 로딩

### Quick Model

In [9]:
from scripts.eval import load_model_and_tokenizer

# 모델 경로 설정
def load_model(model_dir):
    model_path = model_dir
    # 모델과 토크나이저 로드
    model, tokenizer = load_model_and_tokenizer(model_path)
    
    return model, tokenizer

quick_model, tokenizer = load_model(quick_model_data_dir)    

/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Optimized Model

In [10]:
optimized_model, tokenizer = load_model(optimize_model_data_dir)    

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 4. 추론 테스트 및 평가

### 테스트 데이터 셋 로딩

In [11]:
from scripts.eval import get_message_from_dataset, generate_response

article_num =6
messages, full_test_dataset = get_message_from_dataset(sample_dataset_json_file = full_test_data_json, 
                                                       article_num=article_num)    

lenght of full test dataset:  2740
messages: 
 [{'content': 'You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings.', 'role': 'system'}, {'content': 'Please summarize the goals for journalist in this text:\n\n현대자동차 노조가 끝내 파업 절차를 밟기로 뜻을 모았다. 현대차 노조는 1일 전체 조합원 4만6 568명 중 4만958명이 참석한 가운데 2022년 단체협상 결렬에 따른 쟁의행위 찬반투표를 한 결과 3만3 436명 재적 대비 71.8% 이 찬성했다고 밝혔다. 7 435명 재적 대비 15.97% 은 반대표를 던졌다. 오는 4일 중노위로부터 조정중지 결정이 나오면 노조는 합법적으로 파업 등을 할 수 있는 쟁의권을 확보하게 된다. 노조가 파업을 하게 되면 지난 2018년 이후 4년 만이다. 앞서 현대차 노사는 지난 5월 10일 상견례를 시작으로 모두 12차례 교섭을 했지만 이견차를 좁히지 못하고 지난달 22일 교섭 결렬을 선언했다. 현대차 노조는 기본급 16만5 200원 인상 호봉승급분 제외 신규인원 충원과 정년연장 성과급 전년도 순이익의 30% 지급 미래차 공장 국내 신설을 요구했다.', 'role': 'user'}]


In [12]:
# messages, full_test_dataset = get_message_from_dataset(sample_dataset_json_file = full_test_data_json, article_num=175)    

### 추론: Quick Model

In [13]:
%%time
generated_summary, original_summary = generate_response(messages, quick_model, tokenizer, full_test_dataset, 
                                                        max_new_tokens=256, article_num=article_num)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**Query:**
Please summarize the goals for journalist in this text:

현대자동차 노조가 끝내 파업 절차를 밟기로 뜻을 모았다. 현대차 노조는 1일 전체 조합원 4만6 568명 중 4만958명이 참석한 가운데 2022년 단체협상 결렬에 따른 쟁의행위 찬반투표를 한 결과 3만3 436명 재적 대비 71.8% 이 찬성했다고 밝혔다. 7 435명 재적 대비 15.97% 은 반대표를 던졌다. 오는 4일 중노위로부터 조정중지 결정이 나오면 노조는 합법적으로 파업 등을 할 수 있는 쟁의권을 확보하게 된다. 노조가 파업을 하게 되면 지난 2018년 이후 4년 만이다. 앞서 현대차 노사는 지난 5월 10일 상견례를 시작으로 모두 12차례 교섭을 했지만 이견차를 좁히지 못하고 지난달 22일 교섭 결렬을 선언했다. 현대차 노조는 기본급 16만5 200원 인상 호봉승급분 제외 신규인원 충원과 정년연장 성과급 전년도 순이익의 30% 지급 미래차 공장 국내 신설을 요구했다.

**Original Answer:**
현대차 노조는 2022년 단체협상 결렬에 따른 쟁의행위 찬반투표 결과 71.8%가 찬성하여 4년만에 파업절차에 들어가 합법적으로 파업 등을 할 수 있는 쟁의권을 확보하게 되었으며 오는 4일 중노위로부터 조정중지 결정이 나오면 노조는 합법적으로 파업 등을 할 수 있는 쟁의권을 확보하게 된다.

**Generated Answer:**
1. 현대자동차 노조가 파업 절차를 밟기로 뜻을 모았다. 현대차 노조는 1일 전체 조합원 4만6 568명 중 4만958명이 참석한 가운데 2022년 단체협상 결렬에 따른 쟁의행위 찬반투표를 한 결과 3만3 436명 재적 대비 71.8% 이 찬성했다고 밝혔다. 7 435명 재적 대비 15.97% 은 반대표를 던졌다. 오는 4일 중노위로부터 조정중지 결정이 나오면 노조는 합법적으로 파업 등을 할 수 있는 쟁의권을 확보하게 된다. 노조가 파업을 하게 되면 지난 2018년 이후 4년 만이다. 앞서 현대차 

### 평가하기: Quick Model

In [14]:
from scripts.eval import single_evaluate

single_evaluate(generated_summary, original_summary)        

{
  "rouge_scores": {
    "rouge-1": 0.403,
    "rouge-2": 0.317,
    "rouge-l": 0.403
  },
  "bert_score": {
    "precision": 0.711,
    "recall": 0.855,
    "f1": 0.776
  },
  "overall_ro-1-2-l_bert_score": 0.475
}


{'rouge_scores': {'rouge-1': 0.403, 'rouge-2': 0.317, 'rouge-l': 0.403},
 'bert_score': {'precision': 0.711, 'recall': 0.855, 'f1': 0.776},
 'overall_ro-1-2-l_bert_score': 0.475}

### 추론: Optimized Model

In [15]:
%%time
generated_summary, original_summary = generate_response(messages, optimized_model, tokenizer, full_test_dataset, 
                                                        max_new_tokens=256, article_num=article_num)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**Query:**
Please summarize the goals for journalist in this text:

현대자동차 노조가 끝내 파업 절차를 밟기로 뜻을 모았다. 현대차 노조는 1일 전체 조합원 4만6 568명 중 4만958명이 참석한 가운데 2022년 단체협상 결렬에 따른 쟁의행위 찬반투표를 한 결과 3만3 436명 재적 대비 71.8% 이 찬성했다고 밝혔다. 7 435명 재적 대비 15.97% 은 반대표를 던졌다. 오는 4일 중노위로부터 조정중지 결정이 나오면 노조는 합법적으로 파업 등을 할 수 있는 쟁의권을 확보하게 된다. 노조가 파업을 하게 되면 지난 2018년 이후 4년 만이다. 앞서 현대차 노사는 지난 5월 10일 상견례를 시작으로 모두 12차례 교섭을 했지만 이견차를 좁히지 못하고 지난달 22일 교섭 결렬을 선언했다. 현대차 노조는 기본급 16만5 200원 인상 호봉승급분 제외 신규인원 충원과 정년연장 성과급 전년도 순이익의 30% 지급 미래차 공장 국내 신설을 요구했다.

**Original Answer:**
현대차 노조는 2022년 단체협상 결렬에 따른 쟁의행위 찬반투표 결과 71.8%가 찬성하여 4년만에 파업절차에 들어가 합법적으로 파업 등을 할 수 있는 쟁의권을 확보하게 되었으며 오는 4일 중노위로부터 조정중지 결정이 나오면 노조는 합법적으로 파업 등을 할 수 있는 쟁의권을 확보하게 된다.

**Generated Answer:**
1일 현대차 노조는 1일 전체 조합원 4만6 568명 중 4만958명이 참석한 가운데 2022년 단체협상 결렬에 따른 쟁의행위 찬반투표를 한 결과 3만3 436명 재적 대비 71.8% 이 찬성했다고 밝혔으며 7 435명 재적 대비 15.97% 은 반대표를 던졌고 4일 중노위로부터 조정중지 결정이 나오면 노조는 합법적으로 파업 등을 할 수 있는 쟁의권을 확보하게 된다.
CPU times: user 6.7 s, sys: 0 ns, total: 6.7 s
Wall time: 6.69 s


### 평가하기: Optimized Model

In [16]:
from scripts.eval import single_evaluate

single_evaluate(generated_summary, original_summary)        

{
  "rouge_scores": {
    "rouge-1": 0.575,
    "rouge-2": 0.458,
    "rouge-l": 0.575
  },
  "bert_score": {
    "precision": 0.798,
    "recall": 0.882,
    "f1": 0.838
  },
  "overall_ro-1-2-l_bert_score": 0.611
}


{'rouge_scores': {'rouge-1': 0.575, 'rouge-2': 0.458, 'rouge-l': 0.575},
 'bert_score': {'precision': 0.798, 'recall': 0.882, 'f1': 0.838},
 'overall_ro-1-2-l_bert_score': 0.611}

## 5. 다량 테스트 셋을 평가하기

### Quick Model

In [17]:
%%time 

from scripts.eval import SummaryEvaluationSystem

evaluator = SummaryEvaluationSystem(result_folder="eval_result")
summary_json_file, evaluation_results = evaluator.run_evaluation(
                             test_dataset_json=full_test_data_json, 
                             sample_test_num=10, 
                             model_name ="quick_model",
                             model=quick_model, 
                             tokenizer=tokenizer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


output_directory: eval_result/2024-08-26-12-17-53
lenght of full test dataset:  2740


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene


평가 상세 결과가 eval_result/2024-08-26-12-17-53/quick_model_evaluation_results.csv 파일로 저장되었습니다.
JSON 데이터가 'eval_result/2024-08-26-12-17-53/quick_model_summary_results.json' 파일로 저장되었습니다.
평균 평가 점수:
{
  "rouge_scores": {
    "rouge-1": 0.524,
    "rouge-2": 0.457,
    "rouge-l": 0.523
  },
  "bert_score": {
    "precision": 0.758,
    "recall": 0.828,
    "f1": 0.79
  },
  "overall_ro-1-2-l_bert_score": 0.574
}
CPU times: user 2min 2s, sys: 5.35 s, total: 2min 7s
Wall time: 1min 56s


In [18]:
import pandas as pd

eval_result_df = pd.read_csv(evaluation_results)
eval_result_df

,summary_id,rouge-1,rouge-2,rouge-l,bert_precision,bert_recall,bert_f1,overall_score
0,0,0.219,0.109,0.219,0.668,0.744,0.704,0.313
1,1,0.959,0.959,0.959,0.979,0.951,0.965,0.960
2,2,0.400,0.275,0.400,0.752,0.838,0.793,0.467
3,3,0.881,0.879,0.881,0.792,0.820,0.806,0.862
4,4,0.364,0.250,0.364,0.691,0.795,0.740,0.429
5,5,0.800,0.791,0.800,0.958,0.932,0.945,0.834
6,6,0.554,0.525,0.554,0.735,0.907,0.812,0.611
7,7,0.070,0.000,0.056,0.614,0.640,0.627,0.188
8,8,0.443,0.400,0.443,0.730,0.844,0.783,0.517
9,9,0.553,0.385,0.553,0.661,0.813,0.729,0.555


In [19]:
import json

with open(summary_json_file, 'r', encoding='utf-8') as file:
    data = json.load(file)
data    


{'rouge_scores': {'rouge-1': 0.524, 'rouge-2': 0.457, 'rouge-l': 0.523},
 'bert_score': {'precision': 0.758, 'recall': 0.828, 'f1': 0.79},
 'overall_ro-1-2-l_bert_score': 0.574}

### Optimized Model

In [20]:
%%time 

from scripts.eval import SummaryEvaluationSystem

evaluator = SummaryEvaluationSystem(result_folder="eval_result")
summary_json_file, evaluation_results = evaluator.run_evaluation(
                             test_dataset_json=full_test_data_json, 
                             sample_test_num=10, 
                             model_name ="optimized_model",
                             model=optimized_model, 
                             tokenizer=tokenizer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


output_directory: eval_result/2024-08-26-12-20-09
lenght of full test dataset:  2740


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene


평가 상세 결과가 eval_result/2024-08-26-12-20-09/optimized_model_evaluation_results.csv 파일로 저장되었습니다.
JSON 데이터가 'eval_result/2024-08-26-12-20-09/optimized_model_summary_results.json' 파일로 저장되었습니다.
평균 평가 점수:
{
  "rouge_scores": {
    "rouge-1": 0.674,
    "rouge-2": 0.603,
    "rouge-l": 0.655
  },
  "bert_score": {
    "precision": 0.847,
    "recall": 0.861,
    "f1": 0.852
  },
  "overall_ro-1-2-l_bert_score": 0.696
}
CPU times: user 1min 18s, sys: 5.73 s, total: 1min 24s
Wall time: 1min 13s


In [21]:
import pandas as pd

eval_result_df = pd.read_csv(evaluation_results)
eval_result_df

,summary_id,rouge-1,rouge-2,rouge-l,bert_precision,bert_recall,bert_f1,overall_score
0,0,0.976,0.977,0.976,0.991,0.990,0.990,0.980
1,1,0.868,0.753,0.842,0.946,0.938,0.942,0.851
2,2,0.649,0.533,0.649,0.875,0.884,0.879,0.677
3,3,0.791,0.706,0.791,0.844,0.886,0.865,0.788
4,4,0.640,0.625,0.640,0.829,0.949,0.885,0.698
5,5,0.696,0.556,0.696,0.870,0.677,0.762,0.677
6,6,0.841,0.761,0.841,0.944,0.979,0.961,0.851
7,7,0.000,0.000,0.000,0.500,0.560,0.528,0.132
8,8,0.645,0.619,0.645,0.818,0.855,0.836,0.686
9,9,0.632,0.500,0.474,0.855,0.891,0.873,0.620


In [22]:
import json

with open(summary_json_file, 'r', encoding='utf-8') as file:
    data = json.load(file)
data    


{'rouge_scores': {'rouge-1': 0.674, 'rouge-2': 0.603, 'rouge-l': 0.655},
 'bert_score': {'precision': 0.847, 'recall': 0.861, 'f1': 0.852},
 'overall_ro-1-2-l_bert_score': 0.696}